# Python KafkaConsumer

In [ ]:
from kafka import KafkaConsumer 

brokers = ["kafka1:19091", "kafka2:19092", "kafka3:19093"]

# consumer works like a python generator
consumer = KafkaConsumer("airflow-log", bootstrap_servers=brokers) 

for message in consumer:
  print(message)

ConsumerRecord(topic='airflow-log', partition=0, offset=7125, timestamp=1648985727372, timestamp_type=0, key=None, value=b'{"@timestamp":"2022-04-03T11:35:27.372Z","@metadata":{"beat":"filebeat","type":"_doc","version":"8.1.2"},"host":{"name":"deploy-server"},"agent":{"name":"deploy-server","type":"filebeat","version":"8.1.2","ephemeral_id":"38022be4-bad5-4ae2-a94a-eeea5ea81f56","id":"93590cb2-3fe0-40d6-b21c-b1021a053ce6"},"log":{"offset":1073683,"file":{"path":"/root/PySpark/workspace/logs/dag_processor_manager/dag_processor_manager.log"}},"message":"[2022-04-03 11:35:22,009] {manager.py:1069} INFO - Failing jobs without heartbeat after 2022-04-03 11:30:22.009587+00:00","input":{"type":"log"},"ecs":{"version":"8.0.0"}}', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=606, serialized_header_size=-1)
ConsumerRecord(topic='airflow-log', partition=0, offset=7126, timestamp=1648985734373, timestamp_type=0, key=None, value=b'{"@timestamp":"2022-04-03T11:35:34.373Z"

# PySpark Consumer for Structured Streaming

In [1]:
spark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
# from pyspark.sql.functions import *
from pyspark.sql.types import *
import math
import string
import random
import os

In [2]:
# Construct a streaming DataFrame that reads from testtopic
KAFKA_OUTPUT_TOPIC_NAME_CONS = "airflow-log"
KAFKA_BOOTSTRAP_SERVERS_CONS = ["kafka1:19091", "kafka2:19092", "kafka3:19093"]

df  = spark \
    .readStream \
    .format("rate") \
    .option("kafka.bootstrap.servers",KAFKA_BOOTSTRAP_SERVERS_CONS ) \
    .option("subscribe", KAFKA_OUTPUT_TOPIC_NAME_CONS ) \
    .option("startingOffsets", "latest") \
    .option("failOnDataLoss","False") \
    .load()

# df = spark \
#     .readStream \
#     .format("kafka") \
#     .option("kafka.bootstrap.servers",KAFKA_BOOTSTRAP_SERVERS_CONS ) \
#     .option("subscribe", KAFKA_OUTPUT_TOPIC_NAME_CONS ) \
#     .load()
# df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [3]:
ds = df.selectExpr("CAST(value AS STRING)")

# Create output for Spark Structured Streaming

In [4]:
rawQuery = df \
        .writeStream \
        .queryName("qraw")\
        .format("memory")\
        .start()

In [5]:
alertQuery = ds \
        .writeStream \
        .queryName("qalerts")\
        .format("memory")\
        .start()

In [6]:
raw = spark.sql("select * from qalerts")
raw.show()

+-----+
|value|
+-----+
|    0|
|    1|
|    2|
|    3|
|    4|
|    5|
|    6|
|    7|
|    8|
|    9|
+-----+



In [7]:
raw = spark.sql("select * from qraw")
raw.show()

+--------------------+-----+
|           timestamp|value|
+--------------------+-----+
|2022-04-03 05:14:...|    0|
|2022-04-03 05:14:...|    1|
|2022-04-03 05:14:...|    2|
|2022-04-03 05:14:...|    3|
|2022-04-03 05:14:...|    4|
|2022-04-03 05:14:...|    5|
|2022-04-03 05:14:...|    6|
|2022-04-03 05:14:...|    7|
|2022-04-03 05:14:...|    8|
|2022-04-03 05:14:...|    9|
|2022-04-03 05:15:...|   10|
|2022-04-03 05:15:...|   11|
|2022-04-03 05:15:...|   12|
|2022-04-03 05:15:...|   13|
+--------------------+-----+

